In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from scipy import stats
from sklearn.model_selection import KFold

from kerashypetune import KerasRandomSearch, KerasRandomSearchCV

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

In [4]:
def get_model(param):
    
    set_seed(33)
        
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': stats.randint(32, 128),
    'lr': stats.loguniform(1e-4, 0.1), 
    'epochs': 100, 
    'batch_size': 512
}

# Fixed validation search

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model  
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasRandomSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'unit_1': 128, 'unit_2': 52, 'lr': 0.0005565835702925923, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping
SCORE: 0.94634 at epoch 55

***** (2/15) *****
Search({'unit_1': 64, 'unit_2': 65, 'lr': 0.00013052239753580174, 'epochs': 100, 'batch_size': 512})
SCORE: 0.92845 at epoch 100

***** (3/15) *****
Search({'unit_1': 64, 'unit_2': 105, 'lr': 0.0023668178974410024, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
SCORE: 0.93858 at epoch 30

***** (4/15) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.015329273569045341, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
SCORE: 0.95444 at epoch 20

***** (5/15) *****
Search({'unit_1': 64, 'unit_2': 47, 'lr': 0.0681537581148546,

In [7]:
kgs.scores

[0.94634,
 0.92845,
 0.93858,
 0.95444,
 0.90753,
 0.93824,
 0.94803,
 0.94533,
 0.95039,
 0.94094,
 0.93925,
 0.93689,
 0.77759,
 0.94398,
 0.9379]

In [8]:
kgs.best_score

0.95444

In [9]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 37,
 'lr': 0.015329273569045341,
 'epochs': 20,
 'batch_size': 512,
 'steps_per_epoch': 12}

In [10]:
kgs.best_model

# Fixed validation search Generator

In [11]:
gen = ImageDataGenerator(
         rotation_range=90,
         width_shift_range=0.1,
         height_shift_range=0.1,
         zoom_range=0.2)

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasRandomSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(gen.flow(np.expand_dims(x_train,-1), y_train, batch_size=param_grid['batch_size'], seed=33), 
           validation_data=(np.expand_dims(x_test,-1), y_test), 
           callbacks=[es], steps_per_epoch=len(x_train)//param_grid['batch_size'])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'unit_1': 128, 'unit_2': 52, 'lr': 0.0005565835702925923, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping
SCORE: 0.8785 at epoch 56

***** (2/15) *****
Search({'unit_1': 64, 'unit_2': 65, 'lr': 0.00013052239753580174, 'epochs': 100, 'batch_size': 512})
SCORE: 0.77725 at epoch 99

***** (3/15) *****
Search({'unit_1': 64, 'unit_2': 105, 'lr': 0.0023668178974410024, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
SCORE: 0.87108 at epoch 35

***** (4/15) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.015329273569045341, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
SCORE: 0.87344 at epoch 33

***** (5/15) *****
Search({'unit_1': 64, 'unit_2': 47, 'lr': 0.0681537581148546, '

In [13]:
kgs.scores

[0.8785,
 0.77725,
 0.87108,
 0.87344,
 0.32973,
 0.8542,
 0.8974,
 0.85555,
 0.84813,
 0.84475,
 0.784,
 0.69389,
 0.15829,
 0.83598,
 0.84543]

In [14]:
kgs.best_score

0.8974

In [15]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 41,
 'lr': 0.004371872304807245,
 'epochs': 24,
 'batch_size': 512,
 'steps_per_epoch': 11}

In [16]:
kgs.best_model

# Cross validation search

In [17]:
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

X.shape, y.shape

((8999, 28, 28), (8999,))

In [18]:
cv = KFold(n_splits=3, random_state=33, shuffle=True)

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)

kgs = KerasRandomSearchCV(hypermodel, param_grid, cv=cv, n_iter=5, sampling_seed=33,
                          monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(X, y, callbacks=[es])


##################
###  Fold 001  ###
##################

5 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/5) *****
Search({'unit_1': 128, 'unit_2': 52, 'lr': 0.0005565835702925923, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00055: early stopping
SCORE: 0.93333 at epoch 45

***** (2/5) *****
Search({'unit_1': 64, 'unit_2': 65, 'lr': 0.00013052239753580174, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00091: early stopping
SCORE: 0.924 at epoch 89

***** (3/5) *****
Search({'unit_1': 64, 'unit_2': 105, 'lr': 0.0023668178974410024, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
SCORE: 0.93967 at epoch 33

***** (4/5) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.015329273569045341, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Ep

In [19]:
kgs.folds_scores

{'fold 1': [0.93333, 0.924, 0.93967, 0.94967, 0.90233],
 'fold 2': [0.94333, 0.92267, 0.947, 0.952, 0.90167],
 'fold 3': [0.94398, 0.93164, 0.94498, 0.95065, 0.89163]}

In [20]:
kgs.folds_best_score

{'fold 1': 0.94967, 'fold 2': 0.952, 'fold 3': 0.95065}

In [21]:
kgs.folds_best_params

{'fold 1': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 13,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 2': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 30,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 3': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 28,
  'batch_size': 512,
  'steps_per_epoch': 12}}

In [22]:
kgs.folds_best_models

{'fold 1': <tensorflow.python.keras.engine.sequential.Sequential at 0x28e8bd74ef0>,
 'fold 2': <tensorflow.python.keras.engine.sequential.Sequential at 0x28e8bce36d8>,
 'fold 3': <tensorflow.python.keras.engine.sequential.Sequential at 0x28e8a4350f0>}

In [23]:
kgs.best_params_score

0.95077

In [24]:
kgs.best_params

[{'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 13,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 30,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.015329273569045341,
  'epochs': 28,
  'batch_size': 512,
  'steps_per_epoch': 12}]